In [ ]:
import numpy as np

import nbimporter
import m01_profiles as profiles
import m02_elections as elections
import m03_weighting as weighting
import m04_issue_voting as voting

# TO DO
- Compute agreements for all election rules given and save data to file
- Compute means and variances of the agreements for every rule
- For each independent variable, create a plot comparing the agreement of each rule under RD
- Do the same for FRD for each delegation type

In [ ]:
def agreements_rd(n_iter, n_voters, n_cands, n_issues, voters_p, cands_p, rules_list):
    agreements = {rule:[] for rule in rules_list}
    
    for i in range(n_iter):
        v_profile, c_profile, vc_distances = profiles.generate_instance(n_voters, 
                                                                        n_cands, 
                                                                        n_issues, 
                                                                        voters_p, 
                                                                        cands_p, 
                                                                        seed=i)
        #print(f'v_profile: {v_profile}')
        #print(f'c_profile: {c_profile}')
        #print(f'vc_distances: {vc_distances}')
        voter_majority_outcomes = voting.majority(v_profile)
#         print(f'voter majority outcomes: {voter_majority_outcomes}')

        derivatives = profiles.all_issues_profile_derivatives(v_profile, c_profile, threshold)

        for rule in rules_list:
            if rule == 'max_approval':
                election_profile = derivatives['threshold_approval_profile']
                rep_ids, _ = elections.max_approval(election_profile, n_reps, seed=i)
            elif rule == 'rav':
                election_profile = derivatives['threshold_approval_profile']
                rep_ids, _ = elections.repeated_alternative_vote(election_profile, n_reps) #no seed, old implementation
            elif rule == 'borda':
                election_profile = derivatives['ordermaps_profile']
                rep_ids, _ = elections.borda(election_profile, n_reps, seed=i)
            elif rule == 'plurality':
                election_profile = derivatives['ordermaps_profile']
                rep_ids, _ = elections.plurality(election_profile, n_reps, seed=i)
            elif rule == 'stv':
                election_profile = derivatives['orders_profile']
                rep_ids = elections.single_transferable_vote(election_profile, n_reps, seed=i)
            elif rule == 'max_score':
                election_profile = derivatives['scores_profile']
                rep_ids, _ = elections.max_score(election_profile, n_reps, seed=i)
            else:
                raise ValueError(f'Rule {rule} not implemented yet in this experiment')
                
            print(f'Using rule: {rule}')

            given_weights = weighting.set_default_given_weights('uniform', election_profile, list(range(n_cands)), rep_ids, normalized = True)
            weights = weighting.aggregate_weights(given_weights)
            issue_outcomes = voting.weighted_majority(c_profile, weights)
            #print(f'weights: {weights}')
            #print(f'disagreements: {(voter_majority_outcomes ^ issue_outcomes)}')
            dd_rd_distance = np.sum(voter_majority_outcomes ^ issue_outcomes) / n_issues
            agreements[rule].append(1-dd_rd_distance)

    return agreements
            
            

# Quick Test

In [ ]:
n_iter = 3
n_voters = 3
n_cands = 5
n_reps = 2
n_issues = 10
threshold = 0.5

voters_p = 0.25
cands_p = 0.5

rules_list = ['max_approval','borda']

outcomes = agreements_rd(n_iter, n_voters, n_cands, n_issues, voters_p, cands_p, rules_list)
print(outcomes)

Using rule: max_approval
here 1
here 2
here 3
cand ids: [0, 1, 2, 3, 4]
rep ids: [3 4]
here 4
here 4
here 4
here 5
Using rule: borda
here 1
here 2
here 3
cand ids: [0, 1, 2, 3, 4]
rep ids: [0 4]
here 4
here 4
here 4
here 5
Using rule: max_approval
here 1
here 2
here 3
cand ids: [0, 1, 2, 3, 4]
rep ids: [2 1]
here 4
here 4
here 4
here 5
Using rule: borda
here 1
here 2
here 3
cand ids: [0, 1, 2, 3, 4]
rep ids: [1 2]
here 4
here 4
here 4
here 5
Using rule: max_approval
here 1
here 2
here 3
cand ids: [0, 1, 2, 3, 4]
rep ids: [2 4]
here 4
here 4
here 4
here 5
Using rule: borda
here 1
here 2
here 3
cand ids: [0, 1, 2, 3, 4]
rep ids: [2 4]
here 4
here 4
here 4
here 5
{'max_approval': [0.6, 0.7, 0.7], 'borda': [0.5, 0.7, 0.7]}
